In [6]:
import os
os.chdir("c:\\Users\\omsha\\OneDrive\\Attachments\\Desktop\\health_chat_bot\\Medical-ChatBot")
print("Current directory:", os.getcwd())



Current directory: c:\Users\omsha\OneDrive\Attachments\Desktop\health_chat_bot\Medical-ChatBot


In [7]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
# extract data from the pdf file
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [9]:
extracted_data=load_pdf_file(data='Data/')

In [10]:
# Split the data into training and Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)


In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings


In [13]:
def download_hugging_face_embeddings(): 
    embeddings=HuggingFaceEmbeddings (model_name='sentence-transformers/all-MiniLM-L6-v2') 
    return embeddings 
# download_hugging_face_embeddings() Embedded Model being used here is Hugging Face 

In [14]:
embeddings = download_hugging_face_embeddings()

C:\Users\omsha\AppData\Local\Temp\ipykernel_16100\957423663.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings (model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\omsha\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [16]:
import os
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY') 
COHERE_API_KEY = os.getenv('COHERE_API_KEY')



In [17]:

from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "healthbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [18]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY  
os.environ["COHERE_API_KEY"] = COHERE_API_KEY


In [19]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)
  

In [20]:
docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)



In [21]:
docsearch

In [ ]:
retriever=docsearch.as_retriever(search_type="similarity",search_kwarg={"k":3})
retrieved_docs=retriever.invoke("What is Acne?")

In [23]:
retrieved_docs=retriever.invoke("What is Acne?")

In [24]:
retrieved_docs

[Document(id='a743a2ec-be51-45ef-954f-ccc8b14f1ab0', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\Medical_Book.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod—A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='a286fbe7-86d6-4c67-a4a6-55ab66818efb', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'modda

In [33]:
from langchain_cohere.chat_models import ChatCohere  
llm = ChatCohere(model="command-r-plus", cohere_api_key=COHERE_API_KEY)
response = llm.invoke("What is Acne?")
print(response)



content='Acne is a common skin condition that occurs when hair follicles become clogged with dead skin cells and oil, leading to the formation of comedones (blackheads and whiteheads) and inflammatory lesions (pimples, cysts, and nodules) on the face, chest, back, and upper arms. It is a multifactorial disease influenced by various factors, including genetics, hormones, bacteria, and inflammation. Acne can have a significant impact on self-esteem and emotional well-being, particularly during adolescence, when it is most prevalent. Effective treatments are available, including topical medications, oral antibiotics, and retinoids, as well as procedural interventions such as chemical peels and laser therapy. Early intervention and appropriate management can help minimize the physical and psychological impact of acne and prevent potential scarring.' additional_kwargs={'id': '2cd088f1-b3d8-4646-95a0-cdbdc7e66bae', 'finish_reason': 'COMPLETE', 'content': 'Acne is a common skin condition that

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# Updated system prompt
system_prompt = (
    "You are an expert medical assistant. "
    "Use the retrieved context below to answer the question accurately. "
    "If you don't find enough information, try to infer based on general knowledge, "
    "but always state the confidence level of your answer.\n\n"
    "{context}"
)

# Create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create a retrieval chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)




In [3]:
# Test the chatbot
response = rag_chain.invoke({"input": "Tell me about acne?"})
print("Bot Answer:", response["answer"])



NameError: name 'rag_chain' is not defined